<a href="https://colab.research.google.com/github/zahyhabibi/TA-HeartRare-Predict/blob/v2/Deep_Neural_Network_for_Heart_Disease_Prediction_4_params.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install Flask pyngrok tensorflow

In [5]:
!pip install Flask-Cors --quiet

In [6]:
# SEL 2: PELATIHAN MODEL & PERSIAPAN FUNGSI

# --- 1. Import Library Utama ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import sys
import os

# --- 2. Muat dan Pahami Data ---
# Pastikan file 'heart.csv' sudah di-upload ke Colab
file_path = '/content/heart.csv'
try:
    df = pd.read_csv(file_path)
    print("✅ Dataset 'heart.csv' berhasil dimuat.")
except FileNotFoundError:
    print("❌ ERROR: File 'heart.csv' tidak ditemukan. Mohon upload file tersebut.")
    sys.exit("Proses berhenti karena dataset tidak ada.")

# Tampilkan ringkasan data
print("\n--- Ringkasan Dataset ---")
print("5 baris pertama data:")
print(df.head())
print("\nInfo kolom:")
df.info()

# --- 3. Pra-pemrosesan Data ---
# Tentukan fitur yang akan digunakan
selected_features = ['Age', 'Sex', 'ChestPainType', 'MaxHR', 'ExerciseAngina']
numerical_features = ['Age', 'MaxHR']
categorical_features = ['Sex', 'ChestPainType', 'ExerciseAngina']

X = df[selected_features]
y = df['HeartDisease']

# Buat preprocessor untuk mengubah data numerik dan kategorikal
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Pisahkan data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Terapkan preprocessor
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)
print("\n✅ Pra-pemrosesan data selesai.")

# --- 4. Bangun dan Latih Model Neural Network ---
input_shape = X_train_transformed.shape[1]

model = Sequential([
    Dense(128, activation='relu', input_shape=(input_shape,)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid') # Output untuk klasifikasi biner
])

# Kompilasi model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print("\n--- Arsitektur Model ---")
model.summary()

# Latih model
print("\n--- Memulai Pelatihan Model ---")
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train_transformed, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping],
    verbose=1
)
print("✅ Pelatihan model selesai.")

# --- 5. Evaluasi Model ---
loss, accuracy = model.evaluate(X_test_transformed, y_test, verbose=0)
print(f"\n--- Hasil Evaluasi Model ---")
print(f"Akurasi pada data uji: {accuracy:.4f}")
print(f"Loss pada data uji: {loss:.4f}")

# --- 6. Definisikan Fungsi untuk Prediksi ---
# Fungsi ini akan dipanggil oleh API untuk memproses input baru
def predict_heart_disease(model, preprocessor, input_data_dict):
    """
    Menerima input berupa dictionary, melakukan pra-pemrosesan,
    dan mengembalikan probabilitas prediksi penyakit jantung.
    """
    # Ubah input dictionary menjadi DataFrame
    input_df = pd.DataFrame([input_data_dict])

    # Lakukan transformasi pada data input
    transformed_input = preprocessor.transform(input_df)

    # Lakukan prediksi dan ubah hasilnya menjadi float standar Python
    prediction_probability = model.predict(transformed_input)[0][0].item()

    return prediction_probability

print("\n🧠 Model dan fungsi prediksi siap digunakan untuk API.")

✅ Dataset 'heart.csv' berhasil dimuat.

--- Ringkasan Dataset ---
5 baris pertama data:
   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR  \
0   40   M           ATA        140          289          0     Normal    172   
1   49   F           NAP        160          180          0     Normal    156   
2   37   M           ATA        130          283          0         ST     98   
3   48   F           ASY        138          214          0     Normal    108   
4   54   M           NAP        150          195          0     Normal    122   

  ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0              N      0.0       Up             0  
1              N      1.0     Flat             1  
2              N      0.0       Up             0  
3              Y      1.5     Flat             1  
4              N      0.0       Up             0  

Info kolom:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,777 (46.00 KB)

 Trainable params: 11,777 (46.00 KB)

 Non-trainable params: 0 (0.00 B)


--- Memulai Pelatihan Model ---
Epoch 1/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.6155 - loss: 0.6624 - val_accuracy: 0.6622 - val_loss: 0.6155
Epoch 2/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.7363 - loss: 0.5294 - val_accuracy: 0.6351 - val_loss: 0.6664
Epoch 3/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8203 - loss: 0.4203 - val_accuracy: 0.6622 - val_loss: 0.7141
Epoch 4/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7977 - loss: 0.4524 - val_accuracy: 0.6757 - val_loss: 0.7033
Epoch 5/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7979 - loss: 0.4537 - val_accuracy: 0.6622 - val_loss: 0.6972
Epoch 6/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8067 - loss: 0.4205 - val_accuracy: 0.6622 - val_loss: 0.7179
Epoch 7/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8175 - loss: 0.4065 - val_accuracy: 0.6622 - val_loss: 0.7225
Epoch 8/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8191 

In [7]:
# SEL 3: JALANKAN API SERVER

# --- 1. Import Library untuk API ---
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from google.colab import userdata
import threading
import time

# --- 2. Definisikan Aplikasi Flask ---
app = Flask(__name__)
CORS(app)  # Mengizinkan akses dari frontend (PENTING!)

# Definisikan endpoint /predict
@app.route('/predict', methods=['POST'])
def handle_prediction():
    """Endpoint untuk menerima data dan mengembalikan prediksi."""
    if not request.is_json:
        return jsonify({"error": "Request harus dalam format JSON"}), 400

    try:
        input_data = request.get_json()
        print(f"✅ Data diterima dari frontend: {input_data}")

        # Panggil fungsi prediksi yang sudah kita buat di sel sebelumnya
        # Variabel 'model' dan 'preprocessor' akan otomatis terbaca dari scope global notebook
        prediction_prob = predict_heart_disease(model, preprocessor, input_data)

        # Kirim balik hasilnya
        return jsonify({'prediction': prediction_prob})

    except Exception as e:
        print(f"❌ Error saat prediksi: {e}")
        return jsonify({"error": f"Terjadi kesalahan internal: {e}"}), 500

# --- 3. Jalankan Flask & ngrok ---
PORT = 5000

# Jalankan Flask di thread terpisah agar tidak mengganggu Colab
def run_flask_app():
    app.run(port=PORT, debug=False)

flask_thread = threading.Thread(target=run_flask_app)
flask_thread.daemon = True
flask_thread.start()
print(f"🚀 Server Flask sedang berjalan di port {PORT}...")
time.sleep(3) # Beri waktu agar server siap

# Hubungkan ke ngrok
try:
    # Ambil token dari Colab Secrets (Wajib di-set sebelumnya)
    ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(ngrok_token)

    # Matikan tunnel lama dan buat yang baru
    ngrok.kill()
    public_url = ngrok.connect(PORT).public_url

    print("=======================================================================")
    print("✅ API Anda sudah ONLINE!")
    print(f"🔗 URL Publik untuk Frontend: {public_url}/predict")
    print("=======================================================================")
    print("Biarkan sel ini tetap berjalan agar API tetap aktif.")

except Exception as e:
    print(f"❌ Gagal menyambungkan ngrok. Pastikan NGROK_AUTH_TOKEN sudah benar di Colab Secrets.")
    print(f"Error: {e}")

🚀 Server Flask sedang berjalan di port 5000...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information abo

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


✅ API Anda sudah ONLINE!
🔗 URL Publik untuk Frontend: https://f671-35-201-238-33.ngrok-free.app/predict
Biarkan sel ini tetap berjalan agar API tetap aktif.
